###### Tasks:

1. Parse info.out files to determine what happened to which planets and when. Chart as a function of time
2. Reading in an ensemble of planetary systems data. Initial and final times, initial vs final semi-major axis distribution (only need to parse first and last lines of file- would throw out planets that die)
    1. what type of graph/chart?

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pylab

data = dict()
planets = []
cols = ['Time','mass','a','e','i','omega','Omega','M','dist_jac','dist_cen','x','y','z','u','v','w']

for run in range (1,101):
    data[run] = dict()
    for sample in range(1,11):
        planet = pd.read_csv(r'/Users/neha/Documents/lanl/hundred all/run'+str(run)+'/PL'+str(sample)+'.el', skiprows=2, delimiter=r"\s+", names=cols) 
        data[run][sample] = dict()
        data[run][sample]['Time'] = [planet['Time'][0], planet['Time'][len(planet['Time']) - 1]]
        data[run][sample]['a'] = [planet['a'][0], planet['a'][len(planet['a']) - 1]]

In [ ]:
runs = {}
labels = ["ejected from system", "solar collision", "planet-planet collision", "surviving planets"]
fates = [0, 0, 0, 0] 

for run in range (1,101):
    numPlanets = 10
    runs[run] = {}
    runs[run]['x'] = [0]
    runs[run]['y'] = [numPlanets]
    
    with open('/Users/neha/Documents/lanl/hundred all/run'+str(run)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                if "ejected" in line:
                    runs[run]['x'].append(float(cols[3]))
                    numPlanets -= 1
                    runs[run]['y'].append(numPlanets)
                    fates[0] += 1
                elif "collided with the central body" in line:
                    runs[run]['x'].append(float(cols[7]))
                    numPlanets -= 1
                    runs[run]['y'].append(numPlanets)
                    fates[1] += 1
                elif "was hit by" in line:
                    runs[run]['x'].append(float(cols[6]))
                    numPlanets -= 1
                    runs[run]['y'].append(numPlanets)
                    fates[2] += 1
                else:
                    print(cols)
        fates[3] += numPlanets

In [ ]:
import numpy as np

for i in range(1, 101):
    run = runs[i]
    f=plt.figure(figsize=(7,4))
    
    plt.xlabel('Time')
    plt.ylabel('Planets left')
    plt.title('Run ' + str(i))
    plt.ylim(0,10)
    
    plt.plot(run['x'], run['y'], color = 'teal')
    
    plt.show()
    # plt.savefig('run' + str(i) + '.png')
    # plt.close

In [ ]:
plt.figure()
plt.pie(fates, labels=labels, autopct='%1.1f%%')
plt.show()